In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.metrics import mean_squared_error, accuracy_score
from ucimlrepo import fetch_ucirepo 

In [4]:
# Fetch dataset 
mushroom = fetch_ucirepo(id=73) 
  
# Save data as X and y variables
X = mushroom.data.features 
y = np.ravel(mushroom.data.targets)

# Expand dataframe columns and look at view dataframe
pd.set_option('display.max_columns', None)
X.head()

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
1,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
2,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
3,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
4,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g


### Encoding Data

In [5]:
# Factorize all columns
for col in X.columns:
    X.loc[:, col] = pd.factorize(X[col], sort = True)[0]

# View first few rows of encoded data
X.iloc[0:5, 0:5]


,cap-shape,cap-surface,cap-color,bruises,odor
0,5,2,4,1,6
1,5,2,9,1,0
2,0,2,8,1,3
3,5,3,8,1,6
4,5,2,3,0,5


In [7]:
# Check for NAs
X.isna().sum()

cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

We are going to randomly assign observations in our dataset to be missing, and then see which imputation methods perform best by comparing their results to our actual dataset. Let's randomly assign NA observations throughout our dataset. We will create a copy of our dataframe and call it `X_na`.

In [8]:
# Create a copy of X variables
X_na = X.copy()

In [9]:
# Assign 10% of new dataframe with NA values
for col in X_na.columns:
    X_na.loc[X_na.sample(frac = 0.1).index, col] = np.nan

In [10]:
# Check to make sure there are missing values
X_na.isna().sum()

cap-shape                   812
cap-surface                 812
cap-color                   812
bruises                     812
odor                        812
gill-attachment             812
gill-spacing                812
gill-size                   812
gill-color                  812
stalk-shape                 812
stalk-root                  812
stalk-surface-above-ring    812
stalk-surface-below-ring    812
stalk-color-above-ring      812
stalk-color-below-ring      812
veil-type                   812
veil-color                  812
ring-number                 812
ring-type                   812
spore-print-color           812
population                  812
habitat                     812
dtype: int64

Now that we have our dataset with missing values, lets impute

In [11]:
# Impute with mode
X_mode_impute = X_na.fillna(X_na.mode().iloc[0])

# Check to make sure there are no NAs
X_mode_impute.isna().sum()

cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [12]:
# Impute with median (using SimpleImputer)
median_impute = SimpleImputer(strategy = 'median')
X_median_impute = median_impute.fit_transform(X_na)
X_median_impute = pd.DataFrame(X_median_impute, columns = X.columns)

# Check to make sure there are no NAs
X_median_impute.isna().sum()

cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [13]:
# Impute with KNN Imputer
knn_impute = KNNImputer(n_neighbors=5)
X_knn_impute = knn_impute.fit_transform(X_na)
X_knn_impute = pd.DataFrame(X_knn_impute, columns = X.columns)

# Check to make sure there are no NAs
X_knn_impute

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,5.0,2.0,6.4,1.0,6.0,1.0,0.0,1.0,4.0,0.0,2.0,2.0,2.0,7.0,7.0,0.0,2.0,1.0,4.0,2.6,3.0,5.0
1,2.0,2.0,9.0,1.0,0.0,1.0,0.0,0.0,4.6,0.0,1.0,2.0,2.0,7.0,7.0,0.0,2.0,1.0,4.0,3.0,2.0,1.0
2,0.0,2.0,8.0,1.0,3.0,1.0,0.0,0.0,5.0,0.0,1.0,2.0,2.0,7.0,7.0,0.0,2.0,1.0,4.0,3.0,2.0,3.0
3,5.0,3.0,8.0,1.0,6.0,1.0,0.0,1.0,5.0,0.0,2.0,2.0,2.0,7.0,7.0,0.0,2.0,1.0,4.0,2.0,3.0,5.0
4,5.0,2.0,3.0,0.0,5.0,1.0,1.0,0.0,4.0,1.0,2.0,0.4,0.8,7.0,7.0,0.0,2.0,1.0,0.0,3.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,3.0,2.0,4.0,0.0,5.0,0.0,0.0,0.0,11.0,0.0,-1.0,2.0,2.0,5.0,5.0,0.0,1.0,1.0,4.0,0.0,1.0,2.0
8120,2.0,2.0,4.0,0.0,5.0,0.0,0.0,0.0,6.4,0.0,-1.0,2.0,2.0,5.0,5.0,0.0,0.0,1.0,4.0,0.0,4.0,2.0
8121,2.0,2.0,4.0,0.0,5.0,0.0,0.0,0.0,5.0,0.0,-1.0,2.0,2.0,5.0,5.0,0.0,1.0,1.0,4.0,0.0,1.0,2.0
8122,3.0,3.0,4.0,0.0,8.0,1.0,0.0,1.0,0.0,1.0,-1.0,2.0,1.0,7.0,6.2,0.0,2.0,1.0,0.0,7.0,4.0,2.0


In [14]:
# Calculate imputation accuracy using mean squared error
mse_mode = mean_squared_error(X, X_mode_impute)
mse_median = mean_squared_error(X, X_median_impute)
mse_knn = mean_squared_error(X, X_knn_impute)

# Report results
print(f"Mode Imputation Performance: {mse_mode}")
print(f"Median Imputation Performance: {mse_median}")
print(f"KNN Imputation Performance: {mse_knn}")

Mode Imputation Performance: 0.46721834295689535
Median Imputation Performance: 0.26860368828611075
KNN Imputation Performance: 0.12186943288124974


It looks like our KNN imputation worked the best! Let'g go ahead and use our imputed data and our real data to run a BAgged Decision Tree

### Random forest classifier with orignial data

In [15]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [16]:
# Number of features to include for tuning

num_features = [1, 4, 7, 10, 13, 16, 19, 22]
accuracy = []

for feature in num_features:
    rf_classifier = RandomForestClassifier(
        n_estimators=50,
        max_depth=3,
        random_state=42,
        max_features=feature    
    )
    rf_classifier.fit(X_train, y_train)

    # Predict and evaluate results

    y_pred = rf_classifier.predict(X_test)
    rf_accuracy = accuracy_score(y_test, y_pred)
    accuracy.append(rf_accuracy)

    print(f"Number of features:{feature}: Random Forest Accuracy: {rf_accuracy}")


Number of features:1: Random Forest Accuracy: 0.916735028712059
Number of features:4: Random Forest Accuracy: 0.9848236259228876
Number of features:7: Random Forest Accuracy: 0.9868744872846595
Number of features:10: Random Forest Accuracy: 0.9835931091058244
Number of features:13: Random Forest Accuracy: 0.9823625922887613
Number of features:16: Random Forest Accuracy: 0.9860541427399507
Number of features:19: Random Forest Accuracy: 0.9819524200164069
Number of features:22: Random Forest Accuracy: 0.9577522559474979


### Random Forest Classifier with imputed data

In [19]:
# Split imputed data
X_train, X_test, y_train, y_test = train_test_split(X_knn_impute, y, test_size=0.3, random_state=42)

In [20]:
# Number of features to include for tuning

num_features = [1, 4, 7, 10, 13, 16, 19, 22]
accuracy = []

for feature in num_features:
    rf_classifier = RandomForestClassifier(
        n_estimators=50,
        max_depth=3,
        random_state=42,
        max_features=feature    
    )
    rf_classifier.fit(X_train, y_train)

    # Predict and evaluate results

    y_pred = rf_classifier.predict(X_test)
    rf_accuracy = accuracy_score(y_test, y_pred)
    accuracy.append(rf_accuracy)

    print(f"Number of features:{feature}: Random Forest Accuracy: {rf_accuracy}")

Number of features:1: Random Forest Accuracy: 0.9163248564397046
Number of features:4: Random Forest Accuracy: 0.9819524200164069
Number of features:7: Random Forest Accuracy: 0.9872846595570139
Number of features:10: Random Forest Accuracy: 0.9872846595570139
Number of features:13: Random Forest Accuracy: 0.9876948318293683
Number of features:16: Random Forest Accuracy: 0.9856439704675964
Number of features:19: Random Forest Accuracy: 0.9811320754716981
Number of features:22: Random Forest Accuracy: 0.9585726004922067
